In [10]:
import psycopg2
from scipy.stats.stats import pearsonr
import csv
import time

#PARAMETROS
debug = True
start_timer = time.time()

#SIMULATION PARAMETERS
pDiscarded = [100,200]         #Avistamientos de MAC maximos permitidos, previo descarte
pVisits = [20,160]              #Tiempo minimo en segundos para ser visita (proxy de entrada a tienda)
pRatio = [0.90, 1.30]   #Ratio de ajuste de visitas (premisa de wifi deshabilitado)
ref_filename = 'visitas_referencia.csv'
sim_result_filename = 'sim_result.csv'
sim_fecha_inicio = '2018-02-01'  # Inclusiva, formato: YYYY-MM-DD
sim_fecha_fin = '2018-03-01'  # NO-Inclusiva, formato: YYYY-MM-DD

#BASE DE DATOS SIMULADOR
sim_database = "postgres"            # e.g. "compose"
sim_hostname = "localhost" # e.g.: "aws-us-east-1-portal.4.dblayer.com"
sim_port = "5432"                 # e.g. 11101 
sim_uid = "postgres"        # e.g. "admin"
sim_pwd = "admin"      # e.g. "xxx"
sim_tenant = "multimax"

#BASE DE DATOS TENANT
#dsn_database = "postgres"            # e.g. "compose"
#dsn_hostname = "pbtpsqlprd01.postgres.database.azure.com" # e.g.: "aws-us-east-1-portal.4.dblayer.com"
#dsn_port = "5432"                 # e.g. 11101 
#dsn_uid = "qa_prod@pbtpsqlprd01"        # e.g. "admin" sherpa/parabolt2017.
#dsn_pwd = "qa_prod_parabolt2017!"      # e.g. "xxx"
#tenant = "multimax"     #e.g. multimax


In [11]:
###############################
#CONEXION CON BASE DE DATOS
###############################

#con base simulador
try:
    sim_conn_string = "host="+sim_hostname+" port="+sim_port+" dbname="+sim_database+" user="+sim_uid+" password="+sim_pwd
    print("Connecting to Simulator DB...")
    sim_conn=psycopg2.connect(sim_conn_string)
    print("Connected to Simulator DB!")
except:
    print("ERROR: Unable to connect to Simulator DB!")
    
#con base Tenant
#try:
    #t_conn_string = "host="+dsn_hostname+" port="+dsn_port+" dbname="+dsn_database+" user="+dsn_uid+" password="+dsn_pwd
    #print("Connecting to Tenant DB...")
    #t_conn=psycopg2.connect(t_conn_string)
    #print("Connected to Tenant DB!")
#except:
    #print("ERROR: Unable to connect to Tenant DB!")

Connecting to Simulator DB...
Connected to Simulator DB!


In [12]:
#PSEUDOCODIGO:
#01. Limpiar base simulador
#02. Copiarme location_data (y otras? cuidado timezone!) de tenantDB a simuladorDB
#03. paracada pDiscarded
#04.    limpiar visits()
#05.    setear location_data.processed = false
#06.    poner discarded en valor actual del for
#07.    correr visit_load()
#08.    paracada combinacion pVisits y pRatio
#09.       setear los 2 parametros
#10.       visits_per_day()
#11.       query resultado y guardarlo
#12. Plotear resultados con matplotlib

###############
#RUN THIS TO SEE RESULTS
def print_sim_results():
    return;

###############
# MAIN
def main_sim_function():
    try:
        global start_timer
        start_timer = time.time()
        sim_result_list = []
        print("Comenzando Simulador", flush=True)

        print("Paso 1: levantando csv con visitas de referencia..", flush=True)
        ref_list = load_visits_referencia()
        print(str("Paso 0 completado! (se levantaron " + str(len(ref_list)) + " días)"), flush=True)

        print("###SKIPPED - Paso 2: copiando location data de tenant a simulador..", flush=True)
        #limpiar_LocationData()
        #copiar_location_data()
        #print("Paso 2 completado!")

        cantE = len(pDiscarded) * len(pVisits) * len(pRatio)
        countE = 0
        print("Paso 2: Comenzando simulacion, total escenarios: " + str(cantE), flush=True)
        for iDiscarded in pDiscarded:
            if debug : printDebug("discarded seteado en : " + str(iDiscarded))
            limpiar_visitas()
            if debug : printDebug("limpiando visitas..")
            set_location_processed_false()
            if debug : printDebug("procesadas en false, corriendo visits_load()..")
            set_sim_setting('discarded', str(iDiscarded))
            run_sim_visits_load()
            for iVisit in pVisits:
                if debug : printDebug("visitas seteado en : " + str(iVisit))
                set_sim_setting('visits', str(iVisit))
                for iRatio in pRatio:
                    if debug : printDebug("ratio seteado en : " + str(iRatio))
                    set_sim_setting('ratio', str(iRatio))
                    countE = countE +1
                    print("Ejecutando iteracion " + str(countE) + " de " + str(cantE) + "..", flush=True )
                    lista_visits_per_day = refresh_visits_per_day(sim_fecha_inicio, sim_fecha_fin)
                    if lista_visits_per_day == ref_list:
                        corrP = pearsonr(lista_visits_per_day,ref_list)[0]
                    else:
                        corrP = 0
                    resultString = str(iDiscarded) + ", " + str(iVisit) +  ", " + str(iRatio) + ", " + str(corrP)   
                    sim_result_list.append(resultString)

        print("Paso 3 completado, guardando resultados en CSV..", flush=True)
        with open(sim_result_filename, "w") as output:
            writer = csv.writer(output, lineterminator='\n')
            for val in sim_result_list:
                writer.writerow([val]) 

        print("Simulacion finalizada!" + " - T=" + str(time.time() - start_timer)[0:7] + "s", flush=True)
            
    except Exception as e:
        print(e.__doc__) #mensaje corto del error
        print(e.message) #detalle completo del error
    return;


def printDebug(debugMsg):
    print("   #debug: " +debugMsg+ " ## T=" + str(time.time() - start_timer)[0:7] + "s", flush=True);
    return;


def load_visits_referencia():
    r = []
    f = open(ref_filename)
    csv_f = csv.reader(f)
    for row in csv_f:
        r.append(float(str(row[1])))
    return r;

def refresh_visits_per_day(fechaInicio, fechaFin):
    if debug : printDebug("Comienza Refresh View") 
    sim_query = "REFRESH MATERIALIZED VIEW " + sim_tenant + '."visit_per_day"'
    sim_cursor = sim_conn.cursor()
    sim_cursor.execute(sim_query)    
    sim_conn.commit()   
    if debug : printDebug("Termina Refresh View y comienza select con orderby") 
    sim_tabla = sim_tenant + '."visit_per_day"'
    sim_query = "select count_mac from " + sim_tabla
    sim_query = sim_query + " WHERE startingdate BETWEEN '" + fechaInicio + "'::timestamp AND '" + fechaFin + "'::timestamp" 
    sim_query = sim_query + " ORDER BY startingdate "  
    sim_cursor = sim_conn.cursor()
    sim_cursor.execute(sim_query)
    if debug : printDebug("Termina select con orderby") 
    r = []
    rows = sim_cursor.fetchall()
    for row in rows:
        r.append(float(str(row[0])))
    printDebug("lista con visits_per_day " + str(r))
    return r

def run_sim_visits_load():
    sim_proc = sim_tenant + '."visits_load"'
    sim_cursor = sim_conn.cursor()
    sim_cursor.callproc(sim_proc)
    sim_conn.commit()
    
    if debug:
        sim_tabla = '"' + sim_tenant + '"."visits"'
        sim_query = "SELECT count(*) FROM " + sim_tabla
        sim_cursor = sim_conn.cursor()
        sim_cursor.execute(sim_query)    
        rows = sim_cursor.fetchall()
        printDebug("count de visits " + str(rows[0]))    
           
    
    return;

def set_sim_setting(settingName, settingValue):
    sim_tabla = '"' + sim_tenant + '"."settings"'
    sim_query = "UPDATE " + sim_tabla + " SET " + settingName + "='" + settingValue + "'"
    sim_cursor = sim_conn.cursor()
    sim_cursor.execute(sim_query)    
    sim_conn.commit()    
    return;

def set_location_processed_false():
    sim_tabla = '"' + sim_tenant + '"."location_data"'
    sim_query = "UPDATE " + sim_tabla + " SET processed=false"
    sim_cursor = sim_conn.cursor()
    sim_cursor.execute(sim_query)    
    sim_conn.commit()    
    return;

#DEPRECATED (muy lento)
#def copiar_location_data():
#   t_tabla = '"' + tenant + '"."location_data"'
#    t_query = "select * from " + t_tabla
#    t_query = t_query + " WHERE seentime BETWEEN '" + sim_fecha_inicio +"'::timestamp AND '" + sim_fecha_fin +"'::timestamp" #restar un dia para compensar TZ
#    t_query = t_query + " AND sector notnull "
#    t_cursor = t_conn.cursor()
#    print(t_query)
#    t_cursor.execute(t_query)
#    rows = t_cursor.fetchall()
#    for row in rows:
#        tablaInserts = '"' + sim_tenant + '"."location_data"'
#        sim_query = "INSERT INTO " + tablaInserts + "(secret, version, type, apmac, apfloors, clientmac, seenepoch, seentime, rssi, ssid, manufacturer, os, ipv4, ipv6, lat, lng, unc, x, y, sector, processed, data_origin) VALUES ('"
#        for i in range(0,21):
#            sim_query = sim_query + str(row[i]) + "','"             
#        sim_query = sim_query + str(row[21]) + "');" #despues del for, termina el insert
#        try:
#            sim_cursor = sim_conn.cursor()
#            sim_cursor.execute(sim_query)
#            sim_conn.commit()
#            print('.', end='', flush=True)
#        except Exception as e:
#            print(e.__doc__) #mensaje corto del error
#            print(sim_query)
#            raise e
#    return;
        
def limpiar_visitas():
    tabla = sim_tenant + '."visits"'
    query = "DELETE FROM " + tabla
    sim_cursor = sim_conn.cursor()
    sim_cursor.execute(query)
    sim_conn.commit()
    tabla = sim_tenant + '."visits_tmp"'
    query = "DELETE FROM " + tabla
    sim_cursor = sim_conn.cursor()
    sim_cursor.execute(query)
    sim_conn.commit()
    return;

def limpiar_LocationData():
    tabla = sim_tenant + '."location_data"'
    query = "DELETE FROM " + tabla
    sim_cursor = sim_conn.cursor()
    sim_cursor.execute(query)
    sim_conn.commit()
    return;


In [7]:
main_sim_function()

Comenzando Simulador
Paso 1: levantando csv con visitas de referencia..
Paso 0 completado! (se levantaron 28 días)
###SKIPPED - Paso 2: copiando location data de tenant a simulador..
Paso 2: Comenzando simulacion, total escenarios: 8
   #debug: discarded seteado en : 100 ## T=0.01504s
   #debug: limpiando visitas.. ## T=0.01804s
   #debug: procesadas en false, corriendo visits_load().. ## T=42.2725s
   #debug: count de visits (0,) ## T=194.415s
   #debug: visitas seteado en : 20 ## T=194.415s
   #debug: ratio seteado en : 0.9 ## T=194.431s
Ejecutando iteracion 1 de 8..
   #debug: Comienza Refresh View ## T=194.431s
   #debug: Termina Refresh View y comienza select con orderby ## T=194.475s
   #debug: Termina select con orderby ## T=194.478s
   #debug: lista con visits_per_day [] ## T=194.479s
   #debug: ratio seteado en : 1.3 ## T=194.479s
Ejecutando iteracion 2 de 8..
   #debug: Comienza Refresh View ## T=194.483s
   #debug: Termina Refresh View y comienza select con orderby ## T=194.